In [1]:
library(phylofactor)
library("treeio")
library("ggtree")
library("ggplot2")
library(readr)

Loading required package: ape
Loading required package: magrittr
Loading required package: data.table
Loading required package: Matrix

Attaching package: ‘treeio’

The following object is masked from ‘package:ape’:

    drop.tip

ggtree v1.14.6  For help: https://guangchuangyu.github.io/software/ggtree

If you use ggtree in published research, please cite the most appropriate paper(s):

- Guangchuang Yu, David Smith, Huachen Zhu, Yi Guan, Tommy Tsan-Yuk Lam. ggtree: an R package for visualization and annotation of phylogenetic trees with their covariates and other associated data. Methods in Ecology and Evolution 2017, 8(1):28-36, doi:10.1111/2041-210X.12628

- Guangchuang Yu, Tommy Tsan-Yuk Lam, Huachen Zhu, Yi Guan. Two methods for mapping and visualizing associated data on phylogeny using ggtree. Molecular Biology and Evolution 2018, accepted. doi: 10.1093/molbev/msy194

Attaching package: ‘ggtree’

The following object is masked from ‘package:Matrix’:

    expand

The following ob

# Load Data

MODIFY THE PATH OF Microbiome-Factorization

In [ ]:
ROOT <- "/Users/earmingol/Dropbox/Universidad/UCSanDiego/Lab_Knight/Microbiome-Factorization/"

In [ ]:
INPUTS_ <- paste(ROOT, "/outputs/PhyloFactor/", sep='')
OUTPUTS_ <- INPUTS_ # You can Modify this by a completely different path

In [3]:
Taxonomy_filename <- paste(ROOT, "/data/GreenGenes/97_otu_taxonomy.txt", sep='')

# Read taxonomy
taxonomy_ <- read.csv(Taxonomy_filename, header=TRUE, sep="\t", check.names=FALSE)
colnames(taxonomy_) <- c("id", "taxonomy")

Plot parameters

In [2]:
# Phenotypes to plot
phenotypes_ <- c('PhClasses')

# PhyloFactor stuff
data_type_ = "16S_"
ncores <- 3
factors_ <- 40
fam_ <- 'binomial' # Family for model in phylofactor (glm by default)
max_var_ <-'F'
phenotype_type_ <- "Categorical" # Categorical or Continuous
phenotype_bin_size <- 5  # Used only for Continuous phenotype. Bin size to convert phenotype to Categorical.

# Plotting setups
dpi <- 150 # For saving plots
tree_size <- 0.05 # Tree branches thickness
tree_labelsize <- 2 # Tipslabels


Make plots

In [4]:
for (phen in 1:length(phenotypes_)){
    # Load RData files
    INPUTS <- INPUTS_
    phenotype <- phenotypes_[phen]
    load(file=paste(INPUTS, data_type_, phenotype, factors_, ".Rdata", sep=""))
    print(paste(phenotype, "was correctly loaded"))
    
    # Replace variables in case they exist in the Rdata file
    OUTPUTS <- OUTPUTS_
    phenotype <- phenotypes_[phen]
    MetaData <- train_MetaData
    taxonomy <- taxonomy_
    factors <- factors_
    fam <- fam_
    max_var <- max_var_
    phenotype_type <- phenotype_type_
    data_type <- data_type_
    
    # Original Tree
    png(paste(OUTPUTS, data_type, phenotype, factors, "_Original_Tree.png", sep=''), res=dpi,  width=1000, height=1000)
    tax <- as.vector(subset(taxonomy, id %in% filtered_tree$tip.label)[,'taxonomy'])
    new_tax <- c()
    for (a in 1:length(tax)){
        tax_names <- unlist(strsplit(tax[a], "\\;"))
        new_name <- paste(substring(tax_names[2], 5), substring(tax_names[3], 5), sep=' ')
        new_tax <- c(new_tax, new_name)
    }
    
    filtered_tree$tip.label = new_tax
    original_tree <- ggtree(filtered_tree, size=tree_size) + geom_tiplab(size=tree_labelsize)
    plot(original_tree)
    dev.off()
    
    # Factorized tree
    png(paste(OUTPUTS, data_type_, phenotype, factors, "_Factorized_Tree.png", sep=''), res=dpi,  width=1000, height=1000)
    tax2 <- subset(taxonomy, id %in% pf_PhyloFactor$tree$tip.label)
    sorter <- as.vector(unlist(pf_PhyloFactor$tree$tip.label))
    tax2 <- tax2[factor(tax2$id, levels=sorter), ] 
    new_tax <- c()
    for (a in 1:dim(tax2)[1]){
            tax_names <- unlist(strsplit(as.vector(tax2[a, 'taxonomy']), "\\;"))
            new_name <- paste(substring(tax_names[2], 5), substring(tax_names[3], 5), sep=' ')
            new_tax <- c(new_tax, new_name)
        }
    tax2[,'taxonomy'] <- new_tax
    gtree <- pf.tree(pf_PhyloFactor, size=tree_size)
    gtree2 <- ggtree::rotate_tree(gtree$ggplot,-45) %<+% tax2 + geom_tiplab(size=tree_labelsize, aes(label=factor(taxonomy), angle=angle))
    plot(gtree2, xpd = NA)
    dev.off()
    
    # Heatmap
    png(paste(OUTPUTS, data_type, phenotype, factors, "_Heatmap.png", sep=''), res=dpi, width=1000, height=1000)
    hm <- pf.heatmap(pf_PhyloFactor,factors=1:factors,column.order = order(MetaData[,phenotype]))
    plot(hm)
    dev.off()
    
    # Boxplot
    png(paste(OUTPUTS, data_type, phenotype, factors, "_Factor_Boxplots.png", sep=''), res=dpi,  width=1000, height=1000)
    par(mfrow=c(2, 5)) #, mar=c(1,1,1,1))
    x <- pf_PhyloFactor$X[,phenotype]
    
    
    if (phenotype_type == "Continuous"){
            MetaData$CatMeta <-cut(MetaData[,phenotype], seq(from=min(MetaData[,phenotype]), to=max(MetaData[,phenotype]), length.out = phenotype_bin_size))
            unique_phenotypes <- sort(levels(MetaData$CatMeta))
        } else if (phenotype_type == "Categorical") {
            MetaData$CatMeta <- MetaData[,phenotype]
            unique_phenotypes <- sort(unique(MetaData[,phenotype]))
        } else {
             stop('PHENOTYPE_TYPE must be Continuous or Categorical')
        }
    
    # Only ten factors
    for (i in 1:10){
        y <- t(pf_PhyloFactor$basis[,i]) %*% log(pf_PhyloFactor$Data)
        tmp_name = paste('FACTOR', i, sep='')
        MetaData[,tmp_name] = t(y)
        # Initialize val
        elements <- rownames(MetaData[MetaData$CatMeta == unique_phenotypes[1], ])
        val <- list(y[, elements[elements != 'NA']])
        #Continue filling val
        for (j in 2:length(unique_phenotypes)){
            elements <- rownames(MetaData[MetaData$CatMeta == unique_phenotypes[j], ])
            val[[j]] <- y[, elements[elements != 'NA']]
        } 
        boxplot(val,col=gtree$legend$colors[i], xlab="" ,ylab='ILR abundance',
                    main=paste('Factor',i), xaxt="none")
        #mtext(phenotype, side=3)
        axis(1, at=1:length(unique_phenotypes), labels=unique_phenotypes, las=2, cex=0.2)
    }
    title(phenotype, line = -1, outer = TRUE, cex=3.5)
    dev.off()
    
    # Print Factors
    sink(paste(OUTPUTS, data_type, phenotype, factors, ".txt", sep=""))
    for (i in 1:factors){
        cat(paste('\nFACTOR', i, ':\t', pf.taxa(pf_PhyloFactor,taxonomy,factor=i)$group1))
        #cat("\n")
    }
    sink()
}

[1] "PhClasses was correctly loaded"


Coordinate system already present. Adding new coordinate system, which will replace the existing one.
